## This notebook runs a simulation of a rope using ANCF elements to model the flexibility and the DCA to form and solve the equations of motion

In [1]:
import math
import pickle 
import numpy as np
import scipy as sp
import sympy as sym
import matplotlib.pyplot as plt
import MBstructs as MB
import MultiBodyFuncts as MBF
import dca_recursive

from numpy.linalg import inv
from scipy.integrate import odeint

%matplotlib inline

TabError: inconsistent use of tabs and spaces in indentation (MBstructs.py, line 12)

### Classes and Functions

### Physical and Material Properties

In [ ]:
# number of bodies (elements)
n = 12

# Physical Properties
A   =  0.0018
I   =  1.215e-8
L   =  1.2
l   =  L/n

# Material Properties
E   =  0.7e6
rho =  5540

### Initial Conditions

In [ ]:
# start the rope from the horizontal 
theta = 0

# Compute initial nodal coordinates
"""
    Position of any point on the beam initially (straight rope) is 
    r_x = x*cos(theta)
    r_y = x*sin(theta)
    diff(r_x,x) = cos(theta)
    diff(r_y,x) = sin(theta)
"""
# e = [r1x r1y dr1xdx dr1ydx r2x r2y dr2xdx dr2ydx]
state = np.array([[(i*l-l)*np.cos(theta), (i*l-l)*np.sin(theta),
                       np.cos(theta),         np.sin(theta),
                  (i*l)*np.cos(theta),   (i*l)*np.sin(theta),
                       np.cos(theta),         np.sin(theta)] 
             for i in range(1,n+1)])

### Initilize the bodies and joints of the system (compute inertial properties)

In [ ]:
# Create a list of Bodies and Joints
bodies = [MB.ANCF_Element(A, E, I, rho, l, state) for i,state in zip(range(n), state)] # (element)
# This is needed to use the standard O-DCA assembly and disassembly operations 
joints = [MB.Joint(np.array((0, 0, 0, 0, 0)),0) for i in range (n)] # (nodes)

In [ ]:
# Helper Function
#-------------------------
#This function provides a buffer between the dca algorithm and the 
#integrator call. 
def funct(y, t, n, elements, joints, BC1, BC2):

    # update state depedent quantities  
    initialize(bodies, y, n)
    
    # DCA returns the eddot vector
    eddot = recursiveDCA(n,0,bbs,jjs,BC1,BC2,state)
    
    # add the velocities to the state vector and return to the integrator
    ydot = eddot
    ydot[:n] = y[n:]
    return ydot 

### Integration

In [ ]:
# scipy.integrate.odeint is the numerical integrator used
y = odeint(funct,e0,t,(n,0,elements,joints,2,1))

In [ ]:
# Energy Calculation
#--------------------
#The energy of the system is calculated and plotted

energy=MBF.PendEnergy(y,elements)
KE=energy[:,0]
PE=energy[:,1]
TE=energy[:,2]

plt.plot(t,TE-TE[0])
plt.xlabel("Time [s]")
plt.ylabel("energy")
plt.title("System Energy")
plt.show()

plt.plot(t,PE,Time,KE)
plt.xlabel("Time[s]")
plt.ylabel("energy")
plt.title("Kinetic and Potential Energy")
plt.show

In [ ]:
# Solution Plot
#--------------------
plt.plot(Time,yy[:,:n])
plt.xlabel("Time [s]")
plt.ylabel("Generalized Coordinates [Rad]")
plt.title("System Response")

plt.show()

plt.plot(Time,yy[:,n:])
plt.xlabel(("Time[s]"))
plt.ylabel(("Generalized Speeds [Rad/s]"))
plt.title("System Response")

plt.show()

In [ ]:
25%2

In [ ]:
import math
print(math.trunc(40/2))
print(40//2)